In [4]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
#from numpy import *
import numpy as np
import pymongo
%pylab inline

ImportError: No module named pandas

In [5]:
import sys  
for x in sys.path: print x


/Users/yikaluo/anaconda/envs/py27/lib/python27.zip
/Users/yikaluo/anaconda/envs/py27/lib/python2.7
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/plat-darwin
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/plat-mac
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/plat-mac/lib-scriptpackages
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/lib-tk
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/lib-old
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/lib-dynload
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/site-packages/setuptools-20.2.2-py2.7.egg
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/site-packages
/Users/yikaluo/anaconda/envs/py27/lib/python2.7/site-packages/IPython/extensions
/Users/yikaluo/.ipython


In [ ]:
def mongoConnection():
    return pymongo.MongoClient().HousingListings.listings

In [ ]:
res = mongoConnection().find({"sqft": {"$exists": True}, 
                              "bathrooms": {"$exists": True},
                              "latitude": {"$exists": True},
                              "bedrooms": {"$exists": True},
                              "longitude": {"$exists": True},
                              "numImages": {"$exists": True},
                              "description": {"$exists": True},
                              "postingDate": {"$exists": True}
                             })

In [ ]:
#Find price of nearby apartments - average of apartments within 5 blocks.
#Find price of 
len(list(res))

In [ ]:
res[450]

In [ ]:
import geopy
from geopy.distance import vincenty
import datetime

now = datetime.datetime.now()
def nowfn():
    global now
    return now
    

def get_distance(lat, lon):
    center = (37.872105, -122.259470)
    return vincenty(center, (lat, lon)).miles 

def featurize(listing):
    
    return [
                listing["bedrooms"], listing['bathrooms'],
                listing["sqft"], get_distance(listing['latitude'], listing['longitude']),
                listing['numImages'], len(listing['description'].split(" ")),
                (nowfn() - datetime.datetime.fromtimestamp(listing['postingDate'])).days
           ]

In [ ]:
nowfn()


In [ ]:
featurize(res[1000])

In [ ]:
def featurized(cursor):
    x = []
    y = []
    for listing in cursor:
        x.append(featurize(listing))
        y.append(listing['price'])
    return np.array(x), np.array(y)

In [ ]:
x, y = featurized(res)

In [ ]:
from sklearn.svm import SVR


In [ ]:
model.predict(featurize(res[450]))

In [ ]:
def pullListings():
    return list(mongoConnection().find({"sqft": {"$exists": True}, 
                              "bathrooms": {"$exists": True},
                              "latitude": {"$exists": True},
                              "bedrooms": {"$exists": True},
                              "longitude": {"$exists": True},
                              "numImages": {"$exists": True},
                              "description": {"$exists": True},
                              "postingDate": {"$exists": True}
                             }))

def partition(samples, trainsize, valsize):
    permute = np.random.permutation(samples)
    return permute[:valsize], permute[len(permute)-trainsize:]

def train(samples, model):
    x, y = featurized(samples)
    model.fit(x,y)
    return model

def pipeline(listings, model, valsize=500, trainsize=1474-500):
    validation, training = partition(listings, valsize, trainsize)
    model = train(training, model)
    return featurized(validation), featurized(training), model
    
    

In [ ]:
validation, training, model = pipeline(pullListings(), SVR())

val_feats, val_labels = validation
train_feats, train_labels = training

In [ ]:
print(model.predict(val_feats[12]))
print val_labels[12]

In [ ]:
def rss(preds, trues):
    return np.sum((preds-trues)**2)

def gen_predictions(model, val_set):
    return model.predict(val_set)

def rssFromModel(model , val_set, true_labels):
    preds = gen_predictions(model, val_set)
    return rss(preds, true_labels)
    
def plotPipe(model, train_sizes=[2, 100, 200, 300, 400, 500, 600, 700, 800, 974]):
    listings = pullListings()
#     for train
#     validation, training, model = pipeline(pullListings(), SVR())
#     val_feats, val_labels = validation
#     train_feats, train_labels = training
    error = []
    validation, training = partition(listings, len(listings)-500, 500)
    val_feats, val_labels = featurized(validation)
    for trainsize in train_sizes:
        training_n = partition(training, trainsize, 0)[1]
        model = train(training_n, model)
#        training = featurized(training)
        
#         train_feats, train_labels = training
        c = rssFromModel(model, val_feats, val_labels)
        print str(trainsize) + ": " + str(c)
        error.append(c)
    plt.plot(train_sizes, error)
    plt.xlabel("Number of training samples")
    plt.ylabel("Residual sum of squared error")
    plt.show()

In [ ]:
plotPipe(SVR())

In [ ]:
# mod = SVR(C=0.1, kernel='linear')
# plotPipe(mod)

In [ ]:
from sklearn.kernel_ridge import KernelRidge

mod = KernelRidge(alpha=.1, kernel="linear")
plotPipe(mod)

In [ ]:
mod = KernelRidge(alpha=0.05, kernel="rbf")
plotPipe(mod)

In [ ]:
mod = KernelRidge(alpha=0.1, kernel="poly")
plotPipe(mod)

In [ ]:
final = KernelRidge(alpha=.1, kernel="linear")
l = pullListings()
feats, labels = featurized(l)
final.fit(feats, labels)

In [ ]:
import requests
from lxml import html
import time

def linkToVector(link):
    
        r = requests.get(link)
        response = tree = html.fromstring(r.text)
        
        item = {}
        curr = response.xpath("//*[@id='pagecontainer']/section/section/div[2]/p[2]/time/text()")[0].split()[0]
        item["postingDate"] = int(time.mktime(datetime.datetime.strptime(curr, "%Y-%m-%d").timetuple()))
        #pre-processing for getting the price in the right format
        #item["area"] = ''.join(temp.xpath("span")[2].xpath("span[@class='pnr']").xpath("small/text()").extract())
        item["price"] = int(response.xpath("//*[@id='pagecontainer']/section/h2/span[2]/span[1]/text()")[0].replace("$",""))#price.replace("$","")

        maplocation = response.xpath("//div[contains(@id,'map')]")
        latitude = ''.join(maplocation[0].xpath('@data-latitude'))
        longitude = ''.join(maplocation[0].xpath('@data-longitude'))
        tmp = response.xpath("//*[@id='pagecontainer']/section/section/div[1]/div[1]/div[2]/text()")
        if len(tmp) > 0:
            item['address'] = tmp[0]
        if latitude:
            item['latitude'] = float(latitude)
        if longitude:
            item['longitude'] = float(longitude)
        #attr = response.xpath("//p[@class='attrgroup']")
    try:
        item["bedrooms"] = int(response.xpath("//*[@id='pagecontainer']/section/section/div[1]/p[1]/span[1]/b[1]/text()")[0])#int(attr.xpath("span/b/text()")[0])
        #bath = attr.xpath("span/b/text()")[1]
        item["sqft"] = int(response.xpath("//*[@id='pagecontainer']/section/section/div[1]/p[1]/span[2]/b/text()")[0])
        #int(''.join(attr.xpath("span")[1].xpath("b/text()")))
        #if(bath.isdigit()):
        item["bathrooms"] = int(response.xpath("//*[@id='pagecontainer']/section/section/div[1]/p[1]/span[1]/b[2]/text()")[0])#int(attr.xpath("span/b/text()")[1])
         except:
            pass
        item['description'] = "".join(response.xpath("//section[@id='postingbody']/text()"))#.xpath("text()")) 
        item["numImages"] = len(response.xpath("//div[@id='thumbs']/a"))#.xpath("a"))
        return featurize(item)


In [ ]:
linkToVector("https://sfbay.craigslist.org/eby/apa/5507931799.html")

In [ ]:
def predictionFromLink(link, model):
    vector = linkToVector(link)
    return model.predict(vector)

predictionFromLink("https://sfbay.craigslist.org/eby/apa/5507931799.html", final)

In [ ]:
predictionFromLink("https://sfbay.craigslist.org/eby/apa/5517648696.html", final)